In [ ]:
import pandas as pd
xrefs_file="../ontology/xref-all.tsv"
obsoletes_file="../ontology/obsoletes_ordo.tsv"
diff_assertions_file="../ontology/reports/report-all-properties.tsv"
df_xrefs=pd.read_csv(xrefs_file,sep="\t")
df_ordo=pd.read_csv(obsoletes_file,sep="\t")
df_ordo['deprecated']=True
df_ordo.head()

In [ ]:
df_xrefs[~df_xrefs['?deprecated'].isna()].head()

In [ ]:
df_xrefs_ordo=df_xrefs[(df_xrefs['?prefix']=="Orphanet")]
df_xrefs_ordo = df_xrefs_ordo.merge(df_ordo[['?curie','deprecated']], left_on=['?xref'], right_on=['?curie'])
print(len(df_xrefs_ordo))
df_xrefs_ordo.head()

In [ ]:
df_xrefs_ordo.to_csv("../ontology/ordo_deprecated.tsv", sep="\t")

# SSSOM Files

In [ ]:
import pandas as pd
from pathlib import Path
from argparse import ArgumentParser

def get_dataframe(tsv):
    try:
        df = pd.read_csv(tsv,sep="\t", comment="#")
        df["source"]=Path(tsv).stem
        return df
    except pd.errors.EmptyDataError:
        print("WARNING! ", tsv, " is empty and has been skipped.")

sssom_tsv_path = "/Users/matentzn/ws/mondo/src/ontology/mappings/ordo-omim.sssom.tsv"
df = get_dataframe(sssom_tsv_path)

sssom_files = {}

for index, row in df.iterrows():
    object_id = row["object_id"]
    prefix = object_id.split(":")[0]
    if prefix not in sssom_files:
        sssom_files[prefix] = []
    sssom_files[prefix].append(row)

for prefix in sssom_files:
    if
    print(prefix)

print(df.head())


In [ ]:
ps = "https://omim.org/phenotypicSeriesTitles/all?format=tsv"
mim2gene = "https://omim.org/static/omim/data/mim2gene.txt"
mimTitles = "https://data.omim.org/downloads/Wi95pIjsRfqt4ioi9rEJNQ/mimTitles.txt"
genemap2 = "https://data.omim.org/downloads/Wi95pIjsRfqt4ioi9rEJNQ/genemap2.txt"
morbidmap = "https://data.omim.org/downloads/Wi95pIjsRfqt4ioi9rEJNQ/morbidmap.txt"

In [ ]:
import pandas as pd
df_ps = pd.read_csv(ps,sep="\t")

In [ ]:
df_ps.head()

In [1]:
import pandas as pd
diff_current_file="../ontology/reports/report-2020-06-30-release.tsv"
diff_compare_file="../ontology/reports/report-2019-06-29-release.tsv"
raw=pd.read_csv(diff_current_file,sep="\t")
raw=raw.fillna("None")
raw2=pd.read_csv(diff_compare_file,sep="\t")
raw2=raw2.fillna("None")
replace=dict()
replace['<http://purl.obolibrary.org/obo/IAO_']="IAO:"
replace['<http://purl.obolibrary.org/obo/mondo#']="mondo:"
replace['<http://purl.org/dc/elements/1.1/']="dce:"
replace['<http://purl.org/dc/terms/']="dc:"
replace['<http://www.geneontology.org/formats/oboInOwl#']="oio:"
replace['<http://www.w3.org/1999/02/22-rdf-syntax-ns#']="rdf:"
replace['<http://www.w3.org/2000/01/rdf-schema#']="rdfs:"
replace['<http://www.w3.org/2002/07/owl#']="owl:"
replace['<http://www.w3.org/2004/02/skos/core#']="skos:"
replace['>']=""
raw2

,?term,?property,?value,?p,?v
0,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.geneontology.org/formats/oboInOwl#...,GARD:0008595,<http://www.geneontology.org/formats/oboInOwl#...,Orphanet-shared
1,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.geneontology.org/formats/oboInOwl#...,GARD:0008595,<http://www.geneontology.org/formats/oboInOwl#...,OMIM-shared
2,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.geneontology.org/formats/oboInOwl#...,GARD:0008595,<http://www.geneontology.org/formats/oboInOwl#...,MONDO:equivalentTo
3,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.w3.org/2000/01/rdf-schema#subClassOf>,<http://purl.obolibrary.org/obo/MONDO_0016133>,<http://www.geneontology.org/formats/oboInOwl#...,Orphanet:96
4,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.w3.org/2002/07/owl#equivalentClass>,<http://purl.obolibrary.org/obo/OMIM_277460>,None,None
...,...,...,...,...,...
635874,<http://purl.obolibrary.org/obo/MONDO_0022921>,<http://www.geneontology.org/formats/oboInOwl#...,<http://purl.obolibrary.org/obo/mondo#gard_rare>,None,None
635875,<http://purl.obolibrary.org/obo/MONDO_0022921>,<http://www.geneontology.org/formats/oboInOwl#...,short stature microcephaly heart defect,<http://www.geneontology.org/formats/oboInOwl#...,GARD:0000233
635876,<http://purl.obolibrary.org/obo/MONDO_0022921>,<http://www.w3.org/2002/07/owl#deprecated>,true,None,None
635877,<http://purl.obolibrary.org/obo/MONDO_0700005>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://www.w3.org/2002/07/owl#Class>,None,None


In [2]:
df2=raw2[['?term','?property','?value','?p','?v']].drop_duplicates()
df2['?property'] = df2['?property'].replace(replace, regex=True)
df2['?p'] = df2['?p'].replace(replace, regex=True)

In [3]:
df1=raw[['?term','?property','?value','?p','?v']].drop_duplicates()
df1['?property'] = df1['?property'].replace(replace, regex=True)
df1['?p'] = df1['?p'].replace(replace, regex=True)

In [4]:
df=pd.concat([df1, df2, df2]).drop_duplicates(keep=False)
df.to_csv("check.tsv",sep="\t")

In [5]:
df[['?term','?property']].drop_duplicates().groupby(['?property']).agg(['count'])

,?term
,count
?property,
IAO:0000115,225
IAO:0000231,8
IAO:0100001,103
mondo:excluded_subClassOf,154
mondo:excluded_synonym,1
oio:consider,8
oio:created_by,35
oio:hasAlternativeId,3


In [6]:
df_d=pd.concat([raw, raw2, raw2]).drop_duplicates(keep=False)
df_d

,?term,?property,?value,?p,?v
4,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.geneontology.org/formats/oboInOwl#...,"vitamin E, familial isolated deficiency OF",<http://www.geneontology.org/formats/oboInOwl#...,OMIM:277460
5,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.geneontology.org/formats/oboInOwl#...,"vitamin E, familial isolated deficiency OF",<http://www.geneontology.org/formats/oboInOwl#...,MONDO:Lexical
7,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.geneontology.org/formats/oboInOwl#...,"ataxia, Friedreich-like, with selective vitami...",<http://www.geneontology.org/formats/oboInOwl#...,OMIM:277460
8,<http://purl.obolibrary.org/obo/MONDO_0010188>,<http://www.geneontology.org/formats/oboInOwl#...,"vitamin E, familial isolated deficiency OF; VED",<http://www.geneontology.org/formats/oboInOwl#...,OMIM:277460
395,<http://purl.obolibrary.org/obo/MONDO_0025404>,<http://www.geneontology.org/formats/oboInOwl#...,coronaviral enteritis of turkeys,<http://www.geneontology.org/formats/oboInOwl#...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
...,...,...,...,...,...
642137,<http://purl.obolibrary.org/obo/MONDO_0032882>,<http://www.geneontology.org/formats/oboInOwl#...,HEYN-SPROUL-JACKSON SYNDROME; HESJAS,<http://www.geneontology.org/formats/oboInOwl#...,OMIM:618724
642138,<http://purl.obolibrary.org/obo/MONDO_0032882>,<http://www.geneontology.org/formats/oboInOwl#...,OMIM:618724,<http://www.geneontology.org/formats/oboInOwl#...,MONDO:equivalentTo
642139,<http://purl.obolibrary.org/obo/MONDO_0032882>,<http://www.w3.org/2000/01/rdf-schema#subClassOf>,<http://purl.obolibrary.org/obo/MONDO_0003847>,<http://www.geneontology.org/formats/oboInOwl#...,OMIM:618724
642140,<http://purl.obolibrary.org/obo/MONDO_0032882>,<http://www.w3.org/2002/07/owl#equivalentClass>,<http://purl.obolibrary.org/obo/OMIM_618724>,None,None


In [7]:
df_xref=df_d[((df_d['?property']=="<http://www.geneontology.org/formats/oboInOwl#hasDbXref>") & (df_d['?p']!="None") & (df_d['?v']=="MONDO:equivalentTo"))].copy()
df_xref['source'] = df_xref['?value'].str.split(r":", expand=True)[0]
df_xref=df_xref[['?term','source']].drop_duplicates()
df_xref

,?term,source
1951,<http://purl.obolibrary.org/obo/MONDO_0032883>,OMIM
3394,<http://purl.obolibrary.org/obo/MONDO_0033853>,Orphanet
3960,<http://purl.obolibrary.org/obo/MONDO_0032884>,OMIM
5722,<http://purl.obolibrary.org/obo/MONDO_0032885>,OMIM
6228,<http://purl.obolibrary.org/obo/MONDO_0003036>,EFO
...,...,...
639967,<http://purl.obolibrary.org/obo/MONDO_0016825>,GARD
639971,<http://purl.obolibrary.org/obo/MONDO_0016825>,MESH
639986,<http://purl.obolibrary.org/obo/MONDO_0016825>,OMIM
640055,<http://purl.obolibrary.org/obo/MONDO_0032881>,OMIM


In [8]:
df_xref.groupby(['source']).agg(['count'])

,?term
,count
source,
COHD,5
DC,4
DOID,45
EFO,13
GARD,33
ICD10,3
ICD11,1
ICD9,8


In [ ]:
import pandas as pd
import yaml
from datetime import datetime

import os
from sssom.datamodel_util import read_pandas, read_metadata
mapping_dir="../ontology/mappings/"
sssom_file=os.path.join(mapping_dir,"mondo.sssom.tsv")
metadata_file=os.path.join(mapping_dir,"mondo.sssom.yml")

meta, curie_map=read_metadata(metadata_file)
meta["mapping_date"]=datetime.now().strftime("%Y-%m-%d")
df=read_pandas(sssom_file)
df["match_type"]="HumanCurated"
df.head()

In [ ]:
curie_map

In [ ]:
subject_prefixes=set(df['subject_id'].str.split(':', 1, expand=True)[0])
object_prefixes=set(df['object_id'].str.split(':', 1, expand=True)[0])
#object_prefixes.remove("MONDO")
relations=set(df['predicate_id'])

In [ ]:
relations

In [ ]:
from sssom.parsers import from_dataframe
from sssom.writers import write_tsv
import re
import logging

def reconcile_curie(pre_orig):
    if re.match(r".*__[0-9]$", pre_orig):
        print(f"{pre_orig} non standard prefix, reconciling")
        pre = pre_orig[:-3]
    else:
        pre = pre_orig
    return pre


def replace_temporary_prefixes(prefixes, df):
    for pre_orig in prefixes:
        pre = reconcile_curie(pre_orig)
        if pre!=pre_orig:
            print(f"Replacing temporary curie {pre_orig} with {pre}")
            logging.warning(f"Replacing temporary curie {pre_orig} with {pre}")
            df['object_id'] = df['object_id'].str.replace(pre_orig+":",pre+":")
            df['subject_id'] = df['subject_id'].str.replace(pre_orig+":",pre+":")
    return df

df = replace_temporary_prefixes(subject_prefixes, df)
df = replace_temporary_prefixes(object_prefixes, df)

for pre_subj in subject_prefixes:
    for pre_obj in object_prefixes:
        for rel in relations:
            relpre=rel.split(":")[0]
            relppost=rel.split(":")[1]
            sssom_file=os.path.join(mapping_dir,f"{pre_subj.lower()}_{relppost.lower()}_{pre_obj.lower()}.sssom.tsv")
    
            dfs=df[(df['subject_id'].str.startswith(pre_sub+":")) 
                   & (df['predicate_id'] == rel) 
                   & (df['object_id'].str.startswith(pre_obj+":"))]
            if pre_subj in curie_map and pre_obj in curie_map and len(dfs)>0:
                cm = {pre_sub: curie_map[pre_sub], pre_obj: curie_map[pre_obj], relpre: curie_map[relpre]}
                msdf=from_dataframe(dfs,curie_map=cm,meta=meta)
                write_tsv(msdf=msdf,filename=sssom_file)
                print(f"Writing {sssom_file} complete!")
            else:
                print(f"Not creating {sssom_file} because there is a missing prefix ({pre_subj}, {pre_obj}), or no matches ({len(dfs)} matches found)")
    
